# Refutation Notebook
The objective of this notebook is to compare the ability of refuters to detect the problems in a given set of estimators.
Note:
This notebook makes use of the optional dependencies:
- pygraphviz
- causalml

## Import Dependencies

In [1]:
from dowhy.datasets import linear_dataset
from dowhy import CausalModel
import causalml

## Inspection Parameters
These parameters give us the option of inspecting the intermediate steps to sanity check the steps performed

In [2]:
inspect_datasets = True
inspect_models = True
inspect_identified_estimands = True
inspect_estimates = True
inspect_refutations = True

## Estimator List
We pass a list of strings, corresponding to the estimators of interest

In [3]:
estimator_list = ["backdoor.propensity_score_matching", "backdoor.propensity_score_weighting", "backdoor.causalml.inference.meta.LRSRegressor"]
method_params= [ None, None, { "init_params":{} } ]

## Refuter List
A list of strings, corresponding to each refuter we wish to run

In [4]:
refuter_list = ["bootstrap_refuter", "data_subset_refuter"]

## Create the Datasets

In [5]:
# Parameters for creating the Dataset
TREATMENT_IS_BINARY = True
BETA = 10
NUM_SAMPLES = 1000
NUM_CONFOUNDERS = 5
NUM_INSTRUMENTS = 3
NUM_EFFECT_MODIFIERS = 2

# Creating a Linear Dataset with the given parameters
linear_data = linear_dataset(
            beta = BETA,
            num_common_causes = NUM_CONFOUNDERS,
            num_instruments = NUM_INSTRUMENTS,
            num_effect_modifiers = NUM_EFFECT_MODIFIERS,
            num_samples = NUM_SAMPLES,
            treatment_is_binary = True
        )
# Other datasets come here 


# Append them together in an array
datasets = [linear_data]


## Inspect Data

In [6]:
dataset_num = 1
if inspect_datasets is True:
    for data in datasets:
        print("####### Dataset {}###########################################################################################".format(dataset_num))
        print(data['df'].head())
        print("#############################################################################################################")
        dataset_num += 1

####### Dataset 1###########################################################################################
         X0        X1   Z0        Z1   Z2        W0        W1        W2  \
0  2.296454 -1.105707  0.0  0.872820  1.0 -1.017451 -1.121472  0.618193   
1  1.186473 -1.012574  0.0  0.094944  0.0 -1.996002 -0.868432 -2.147540   
2  0.697345 -0.421525  1.0  0.582726  1.0 -0.287837 -1.625909 -1.624035   
3  0.728708 -0.863882  1.0  0.623053  1.0 -2.926781 -1.116355 -0.367452   
4 -0.801555  0.554153  1.0  0.791654  1.0 -2.985773 -0.083998 -0.072799   

         W3        W4     v0          y  
0 -0.111623  0.609580   True  14.862008  
1 -1.543804 -0.003589  False -12.178909  
2 -1.750058  0.397615   True   2.765377  
3 -0.333454  0.768779   True   5.936453  
4  0.509324  2.133821   True  13.790278  
#############################################################################################################


## Create the CausalModels

In [7]:
models = []
for data in datasets:
    model = CausalModel(
                data = data['df'],
                treatment = data['treatment_name'],
                outcome = data['outcome_name'],
                graph = data['gml_graph']
            )
    models.append(model)

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['v0'] on outcome ['y']


## Inspect Models

In [8]:
model_num = 1
if inspect_models is True:
    for model in models:
        print("####### Model {}#############################################################################################".format(model_num))
        print("Common Causes:",model._common_causes)
        print("Effect Modifiers:",model._effect_modifiers)
        print("Instruments:",model._instruments)
        print("Outcome:",model._outcome)
        print("Treatment:",model._treatment)
        print("#############################################################################################################")
        model_num += 1

####### Model 1#############################################################################################
Common Causes: ['W3', 'W1', 'W2', 'Unobserved Confounders', 'W0', 'W4']
Effect Modifiers: ['X1', 'X0']
Instruments: ['Z2', 'Z0', 'Z1']
Outcome: ['y']
Treatment: ['v0']
#############################################################################################################


## Identify Effect

In [9]:
identified_estimands = []
for model in models:
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    identified_estimands.append(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['W3', 'W1', 'W2', 'Unobserved Confounders', 'W0', 'W4']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:['Z2', 'Z0', 'Z1']


## Identified Estimands

In [10]:
estimand_count = 1
for estimand in identified_estimands:
    print("####### Identified Estimand {}#####################################################################################".format(estimand_count))
    print(estimand)
    print("###################################################################################################################")
    estimand_count += 1

####### Identified Estimand 1#####################################################################################
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                                 
─────(Expectation(y|W3,W1,W2,W0,W4))
d[v₀]                               
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W3,W1,W2,W0,W4,U) = P(y|v0,W3,W1,W2,W0,W4)
### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(y, [Z2, Z0, Z1])*Derivative([v0], [Z2, Z0, Z1])**(-1))
Estimand assumption 1, As-if-random: If U→→y then ¬(U →→{Z2,Z0,Z1})
Estimand assumption 2, Exclusion: If we remove {Z2,Z0,Z1}→{v0}, then ¬({Z2,Z0,Z1}→y)

###################################################################################################################


## Estimate Effect

In [11]:
estimate_list = []
for i in range(len(identified_estimands)):
    for j in range(len(estimator_list)):
        estimate = model.estimate_effect(
                        identified_estimands[i],
                        method_name=estimator_list[j],
                        method_params=method_params[j]
                  )
        estimate_list.append(estimate)

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
/home/tanmay/create-refute-notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
/home/tanmay/create-refute-notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sk

## Estimate Values

In [12]:
estimand_count = 1
if inspect_estimates is True:
    for estimand in estimate_list:
        print("####### Estimand {}#######################################################################################".format(estimand_count))
        print("*** Class Name ***")
        print()
        print(estimand.params['estimator_class'])
        print()
        print(estimand)
        print("########################################################################################################")
        print()
        estimand_count += 1
    

####### Estimand 1#######################################################################################
*** Class Name ***

<class 'dowhy.causal_estimators.propensity_score_matching_estimator.PropensityScoreMatchingEstimator'>

*** Causal Estimate ***

## Target estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                                 
─────(Expectation(y|W3,W1,W2,W0,W4))
d[v₀]                               
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W3,W1,W2,W0,W4,U) = P(y|v0,W3,W1,W2,W0,W4)
### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(y, [Z2, Z0, Z1])*Derivative([v0], [Z2, Z0, Z1])**(-1))
Estimand assumption 1, As-if-random: If U→→y then ¬(U →→{Z2,Z0,Z1})
Estimand assumption 2, Exclusion: If we remove {Z2,Z0,Z1}→{v0}, then ¬({Z2,Z0,Z1}→y)

## Realized estimand
b: y~v0+W3+W1+W2+W0+W4
## Estimate
Value: 11.803018265480263

###################################

## Refute Estimate

In [13]:
refutation_list = []
for estimand in identified_estimands:
    for estimate in estimate_list: 
        for refuter in refuter_list:
            ref = model.refute_estimate(estimand, estimate,method_name=refuter)
            refutation_list.append(ref)

INFO:dowhy.causal_refuters.bootstrap_refuter:All variables required: Running bootstrap adding noise to confounders, instrumental variables and effect modifiers.
INFO:dowhy.causal_refuters.bootstrap_refuter:INFO: The chosen variables are: W3,W1,W2,W0,W4,Z2,Z0,Z1,X1,X0
INFO:dowhy.causal_refuters.bootstrap_refuter:Refutation over 100 simulated datasets of size 1000 each
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v

INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Us

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.

...

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y t

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:do

INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy

INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy

INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W3+W1+W2+W0+W4
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy

## Refutation Values

In [14]:
refuter_count = 1
if inspect_refutations is True:
    for refutation in refutation_list:
        print("####### Refutation {}#######################################################################################".format(refuter_count))
        print("*** Class Name ***")
        print()
        print(refutation.refutation_type)
        print()
        print(refutation)
        print("########################################################################################################")
        print()
        refuter_count += 1

####### Refutation 1#######################################################################################
*** Class Name ***

Refute: Bootstrap Sample Dataset

Refute: Bootstrap Sample Dataset
Estimated effect:11.803018265480263
New effect:11.726840078548117
p value:0.45999999999999996

########################################################################################################

####### Refutation 2#######################################################################################
*** Class Name ***

Refute: Use a subset of data

Refute: Use a subset of data
Estimated effect:11.803018265480263
New effect:11.82215833537287
p value:0.46

########################################################################################################

####### Refutation 3#######################################################################################
*** Class Name ***

Refute: Bootstrap Sample Dataset

Refute: Bootstrap Sample Dataset
Estimated effect:11.20381938129325
Ne